# Telus Visualization Project

In this notebook I explain the data cleaning process that I performed prior to analyzing the data. 

In the first few sections I will read and explore the dataset. My exploration step will reveal that there are several missing data points, in almost every field. These missing data (such as city, state, date_time, and latitude and londitude) need to be identified from the other fields or from a complimentary dataset. An example is identification of state for non-USA cities from the field city using an aditional dataset. My exploration also shows that a text field (duration) needs to be transformed into numeric values. Two new fields (country, and population) need to be added to the dataset. And finaly some format conversion, and dropping of useless fiels will be required.

In the following sections I will provide the codes that I used to perform the required actions mentioned above. Most of these sections are supplemented by few queries to check the results and confirm the correctness of the generated data.

At the end the transformed and summerized table will be saved to be uploaded on a Google Cloud database, and Tableau for visualization and insight extraction.

## Reading the data:

In [1]:
#import the required packages

import pandas as pd
import pandasql as pdql
import numpy as np
import re
import datetime
import pycountry
import durations
import timeit

In [2]:
#Load the data set into a dataframe and display the first 5 rows
df_ufo=pd.read_csv('file name')
df_ufo.head()

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude
0,My wife was driving southeast on a fairly popu...,Chester,VA,2019-12-12T18:43:00,light,5 seconds,Occurred : 12/12/2019 18:43 (Entered as : 12/...,http://www.nuforc.org/webreports/151/S151739.html,My wife was driving southeast on a fairly popu...,2019-12-22T00:00:00,37.343152,-77.408582
1,I think that I may caught a UFO on the NBC Nig...,Rocky Hill,CT,2019-03-22T18:30:00,circle,3-5 seconds,Occurred : 3/22/2019 18:30 (Entered as : 03/2...,http://www.nuforc.org/webreports/145/S145297.html,I think that I may caught a UFO on the NBC Nig...,2019-03-29T00:00:00,41.664800,-72.639300
2,I woke up late in the afternoon 3:30-4pm. I we...,NaN,NaN,NaN,NaN,NaN,Occurred : 4/1/2019 15:45 (Entered as : April...,http://www.nuforc.org/webreports/145/S145556.html,I woke up late in the afternoon 3:30-4pm. I w...,NaN,NaN,NaN
3,I was driving towards the intersection of fall...,Ottawa,ON,2019-04-17T02:00:00,teardrop,10 seconds,Occurred : 4/17/2019 02:00 (Entered as : 04-1...,http://www.nuforc.org/webreports/145/S145697.html,I was driving towards the intersection of fall...,2019-04-18T00:00:00,45.381383,-75.708501
4,"In Peoria Arizona, I saw a cigar shaped craft ...",Peoria,NY,2009-03-15T18:00:00,cigar,2 minutes,Occurred : 3/15/2009 18:00 (Entered as : 03/1...,http://www.nuforc.org/webreports/145/S145723.html,"In Peoria, Arizona, I saw a cigar shaped craft...",2019-04-18T00:00:00,NaN,NaN


In [3]:
#Load the data set containing detailed geographical information into a dataframe and display the first 5 rows 
df_worldcities=pd.read_csv('file name')
df_worldcities.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140


## Query Function:

In [4]:
pysql = lambda q: pdql.sqldf(q, globals())

## Exploration:

In [5]:
df_ufo.dtypes

summary            object
city               object
state              object
date_time          object
shape              object
duration           object
stats              object
report_link        object
text               object
posted             object
city_latitude     float64
city_longitude    float64
dtype: object

In [6]:
df_ufo.shape

(88125, 12)

In [7]:
#Number of missing records in each field:
df_ufo.shape[0]-df_ufo.count()

summary              30
city                234
state              5235
date_time          1187
shape              2498
duration           3171
stats                37
report_link           0
text                 55
posted             1187
city_latitude     16112
city_longitude    16112
dtype: int64

In [8]:
#Percent of missing records in each fieald:
100-(df_ufo.count()/df_ufo.shape[0]*100)

summary            0.034043
city               0.265532
state              5.940426
date_time          1.346950
shape              2.834610
duration           3.598298
stats              0.041986
report_link        0.000000
text               0.062411
posted             1.346950
city_latitude     18.283121
city_longitude    18.283121
dtype: float64

In [9]:
df_ufo.drop([ 'report_link','posted','text'], axis=1)

,summary,city,state,date_time,shape,duration,stats,city_latitude,city_longitude
0,My wife was driving southeast on a fairly popu...,Chester,VA,2019-12-12T18:43:00,light,5 seconds,Occurred : 12/12/2019 18:43 (Entered as : 12/...,37.343152,-77.408582
1,I think that I may caught a UFO on the NBC Nig...,Rocky Hill,CT,2019-03-22T18:30:00,circle,3-5 seconds,Occurred : 3/22/2019 18:30 (Entered as : 03/2...,41.664800,-72.639300
2,I woke up late in the afternoon 3:30-4pm. I we...,NaN,NaN,NaN,NaN,NaN,Occurred : 4/1/2019 15:45 (Entered as : April...,NaN,NaN
3,I was driving towards the intersection of fall...,Ottawa,ON,2019-04-17T02:00:00,teardrop,10 seconds,Occurred : 4/17/2019 02:00 (Entered as : 04-1...,45.381383,-75.708501
4,"In Peoria Arizona, I saw a cigar shaped craft ...",Peoria,NY,2009-03-15T18:00:00,cigar,2 minutes,Occurred : 3/15/2009 18:00 (Entered as : 03/1...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
88120,4 lights in formation over Tempe appear while ...,Tempe,AZ,2019-10-02T20:00:00,formation,3 minutes,Occurred : 10/2/2019 20:00 (Entered as : 10/2...,33.414036,-111.920920
88121,"2 bright star like lights in the NNW skys, ((...",Bolivar,MO,2019-10-02T20:00:00,light,20 seconds,Occurred : 10/2/2019 20:00 (Entered as : 10/0...,37.642200,-93.399600
88122,I just witnessed a ‘Phoenix Lights’ type of fo...,North Port,FL,2019-10-02T20:03:00,formation,20 seconds,Occurred : 10/2/2019 20:03 (Entered as : 10/0...,27.076210,-82.223280
88123,"Witnessed an orange, slow moving light. Was lo...",Black Mountain,NC,2019-10-02T22:00:00,fireball,2 minutes,Occurred : 10/2/2019 22:00 (Entered as : 10/0...,35.605000,-82.313200


* Finding: 

The largest missing percentage belongs to the coordinates, followed by the state. 

Although it is unlikely, let's check whether there is any none-empty latitude/longitude record corresponding to a state with null value:

In [10]:
#Checking wether there are any not-empty latitude/longitude records corresponding to the states with null values
query='select count(report_link) from df_ufo where state IS NULL AND city_latitude IS NOT NULL  '
pysql(query)

,count(report_link)
0,0


In [11]:
#Check if missing state information is included in the other fields
query='select stats, city from df_ufo where city REGEXP "[^a-z A-Z]" AND state IS NULL'
pysql(query)

,stats,city
0,Occurred : 6/6/2019 19:00 (Entered as : 6/6/2...,Caloocan City (Philippines)
1,Occurred : 2/6/2006 04:30 (Entered as : 6/2/2...,Woodford Green (UK/England)
2,Occurred : 12/11/1762 21:00 (Entered as : 12/...,"Lulworth, Dorsetshire (near) (UK/England)"
3,Occurred : 7/10/1968 20:30 (Entered as : 07/1...,Tehran (Iran)
4,Occurred : 5/7/1972 00:00 (Entered as : 5/7/7...,Essex (UK/England)
...,...,...
4637,Occurred : 11/28/2019 05:50 (Entered as : 28/...,Pontypridd (UK/Wales)
4638,Occurred : 9/27/2019 20:00 (Entered as : 09/2...,"Drakenstein, Paarl East (South Africa)"
4639,Occurred : 9/29/2019 16:50 (Entered as : 09/2...,Santo André
4640,Occurred : 9/29/2019 19:35 (Entered as : 09/2...,Tijuana (Mexico)


* Finding: 

As expected, the missing states can not be identified from the coordinates (as those are missing too). By investigation, I noticed that some of the missing states are included in city and/or stats field.
<br>

In fact, out of the 5235 NULL entries for states, 4642 of them correspond to some mixed information in the field city. These records are mostly related to non-USA countries (4559 records).
<br>

If we extract those information, up to %89 (4642/5235*100) of missing states can be identified. Having the city, state, and country, we might be able to identify up to %16 of the %18  missing coordinates.

In [12]:
#Check wether there is any messy entries in state too:
query='select state from df_ufo where state REGEXP "[^a-z A-Z]" '
pysql(query)

,state


In [13]:
#Check wether there is any entries in state with a different format:
query='select  LENGTH(state)  from df_ufo where LENGTH(state)>2;'
pysql(query)

,LENGTH(state)


* Finding: The state entries seem OK.
<br>




### Checking the entries for a sample record with NULL value in city, state, date_time, shape, duration, latitude, and longitude:

In [14]:
print(df_ufo.stats[2])

Occurred : 4/1/2019 15:45  (Entered as : April01.19) Reported: 4/8/2019 9:42:31 PM 21:42 Posted: 4/12/2019 Location: Winnipeg (Canada),  Shape:  Duration:


In [15]:
print(df_ufo.summary[2])

I woke up late in the afternoon 3:30-4pm. I went to have a bath, while shaving my legs i noticed indentations around my left ankel. I t


In [16]:
print(df_ufo.text[2])

I woke up late in the afternoon 3:30-4pm.  I went to have a bath, while shaving my legs i noticed indentations around my left ankel.  I then noticed that they went all the way around the backs of both my legs and up to my thighs.  It scared me because a few nights before I was telling my fiancee that was pretty sure that i was abducted before, he laughed then that was that.  I showed him the indentations and he was shocked.  He suffers from insomnia.  We woke up at the same time.  He only sleeps maybe 4-5 hrs a wk.  He said that he slept all night with me.  The indentations look like my legs were seperated into stirups, or like how a massage chair would hold your legs.  The indents were in rows of 1-1.5 lines from my ankles all the way up to my thighs just below my buttocks.  They were perfectly lined.  They disappeared within 2 hrs of waking up.  I couldnt talk a picture because i had just lost my cell phone.



<br>



### Based on my investigations and the project instruction, these are the data cleaning actions required prior to data analysis:
<br>

*   1- The missing/messy entries of city, state and country (which is a new field to be added) must be extracted from stats where possible. Care must be taken wherever there are cities with the same name in different states. 
<br>
    
*    2- Having identified the city, state, and country, the latitude and longitude must be extracted from a complimentary dataset containing the needed information, using a join method on city+state+country as the unique ID.
<br>
*    3- A good portion of the missing date_time data can be extracted from stats. This field then needs to be converted to a standard format.
<br>
    
*    4- The field duration needs to be converted to seconds. 
* Note:  some missing values in duration and shape could theoritically be identified from the field text. This step will be pursued only if time permits.
<br>
    
*    5- The country entries need to be formated according to ISO format.
<br>
    
*    6- Finally the useless fields (such as report_link, and posted) must be dropped, and the leftover null values must be replaced by "unknown". The transformed data can then be saved into a new csv file.


### Before starting the data cleaning, I need to make a few supplimentary variables:

In [17]:
# Let's convert abbreviated states to full name because in the Worldcities table (which will be used later to find the missing lat/long coordinates) the states and provinces are written in non-abbreviated format.

us_state_abbr = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'}

can_prov_abbr={
  'AB': 'Alberta',
  'BC': 'British Columbia',
  'MB': 'Manitoba',
  'NB': 'New Brunswick',
  'NL': 'Newfoundland and Labrador',
  'NS': 'Nova Scotia',
  'NT': 'Northwest Territories',
  'NU': 'Nunavut',
  'ON': 'Ontario',
  'PE': 'Prince Edward Island',
  'QC': 'Quebec',
  'SK': 'Saskatchewan',
  'YT': 'Yukon'
}

        
us_state_can_prov_abbr=us_state_abbr.copy()
us_state_can_prov_abbr.update(can_prov_abbr)


        
def state_fullname(state_abr,state_dict):
    
        try:
            state_full=state_dict[state_abr]
        except:
            state_full="unknown"
            
        return state_full
    
df_ufo['state'] = df_ufo.apply(lambda row: state_fullname(row['state'],us_state_can_prov_abbr),axis=1)


In [18]:
# I'll clean up the city column

#1-For convinience of text processing I'll first convert all string fields into english and lower case

df_ufo['summary']=df_ufo['summary'].str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
df_ufo['city']=df_ufo['city'].str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
df_ufo['state']=df_ufo['state'].str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
df_ufo['date_time']=df_ufo['date_time'].str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
df_ufo['shape']=df_ufo['shape'].str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
df_ufo['duration']=df_ufo['duration'].str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
df_ufo['stats']=df_ufo['stats'].str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

#Similarly for the world cities dataset (admin_name=Province/State):
df_worldcities['city_ascii']=df_worldcities['city_ascii'].str.lower()
df_worldcities['country']=df_worldcities['country'].str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
df_worldcities['admin_name']=df_worldcities['admin_name'].str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

df_worldcities['city_state_country']=df_worldcities['city_ascii']+" "+df_worldcities['admin_name']+" "+df_worldcities['country']
df_worldcities['city_country']=df_worldcities['city_ascii']+" "+df_worldcities['country']
df_worldcities['city_state']= df_worldcities['city_ascii']+" "+df_worldcities['admin_name']

# I add a country column and a population column to be filled later
df_ufo['country']=""
df_ufo['population']=""
df_ufo['city_corrected']=""
df_ufo['date_time_from_stats']=""



### Now, let's check whether the combination of city, state, and country can uniqly identify the different locations: 

In [19]:
query='SELECT city_state_country, COUNT(city_country) FROM df_worldcities GROUP BY city_country HAVING COUNT(city_country) > 1 '
pysql(query)

,city_state_country,COUNT(city_country)
0,abasolo guanajuato mexico,2
1,abbeville louisiana united states,2
2,aberdeen washington united states,5
3,abilene texas united states,2
4,abington pennsylvania united states,2
...,...,...
1069,yuzawa akita japan,2
1070,zarechnyy penzenskaya oblast russia,2
1071,zebbug ebbu malta,2
1072,zheleznogorsk kurskaya oblast russia,2


In [20]:
query='SELECT city_country, COUNT(city_country) FROM df_worldcities GROUP BY city_country HAVING (COUNT(city_country)>1 AND SUM(lat)<>(AVG(lat)*2)) '
pysql(query)

,city_country,COUNT(city_country)
0,aberdeen united states,5
1,albany united states,4
2,alexandria united states,4
3,alpine united states,3
4,alton united states,3
...,...,...
360,winslow united states,3
361,woodbury united states,5
362,woodstock united states,5
363,wyoming united states,3


In [21]:
query='SELECT city_state_country, COUNT(city_state_country) FROM df_worldcities GROUP BY city_state_country HAVING COUNT(city_state_country)>1 '
pysql(query)

,city_state_country,COUNT(city_state_country)
0,adams township pennsylvania united states,2
1,allegheny township pennsylvania united states,2
2,bethel township pennsylvania united states,2
3,cam ranh khnh ha vietnam,2
4,carroll township pennsylvania united states,3
5,center township pennsylvania united states,2
6,conewago township pennsylvania united states,2
7,derry township pennsylvania united states,2
8,fairview township pennsylvania united states,2
9,franklin township new jersey united states,2


In [22]:
query='SELECT city_state_country, COUNT(city_state_country) FROM df_worldcities GROUP BY city_state_country HAVING (COUNT(city_state_country)>1 AND SUM(lat)<>(AVG(lat)*2)) '
pysql(query)

,city_state_country,COUNT(city_state_country)
0,carroll township pennsylvania united states,3
1,jackson township pennsylvania united states,3
2,penn township pennsylvania united states,3
3,richland township pennsylvania united states,3
4,springfield township pennsylvania united states,4
5,washington township new jersey united states,3
6,washington township pennsylvania united states,4


In [23]:
query='select lat,lng from df_worldcities where city_state_country="jackson township pennsylvania united states" '
pysql(query)

,lat,lng
0,40.3774,-76.3142
1,39.9057,-76.8796
2,41.0080,-75.3578



 * Finding: 
 
The results of above queries show that city+country can not identify the location uniquely (there are 365 similar combinations with different latitudes), but city+state+country can. Although for this combination still there are 6 similar records with different latitudes, but seemingly these are duplicate of same record because their coordinates are very close.

# Now, let's begin the cleaning process:

## Step 1:
### The missing/messy values of city, state and country (new field to be added to the table) must be extracted from stats where possible. 

In [24]:
# Make a dictionary of keys=country, value=cities, for all the world countries. This dictionary will be used to faster look for the city once we identified the country (in the next script).

list_of_countries=df_worldcities.country.unique()
dict_country_city={}
dict_city_state={}
dict_country_city_state={}
for country in list_of_countries:
    list_of_cities=df_worldcities.loc[df_worldcities['country'] ==country, 'city_ascii'].values
    dict_country_city[country]=list_of_cities


In [27]:
# First, clean the mixed entries for which both city and country name exist in the city field:


def find_missing_locationinfo(city_original,stats,state_original,dict_country_city,us_state_abbr,can_prov_abbr,df_worldcities):


    identified_country=""
    pattern = re.compile("[^a-z A-Z]")
    try:
        x=pattern.search(city_original) is not None
    except:
        x=0
        
# for the city mixed entries let's first fix UK-variation issue:['uk/england','uk/scotland',' uk/wales', 'uk/englnd','uk/birmingham','uk/endland','uk/enland'],and also remove ()_,/  
# for the non mixed cities: if we have both the city and the state the location can be identified, otherwise not (because there are some cities with same name in different states): for such records the coordinates, if existed, could be used for state identification, but I have already checked and know that there is no entry with null state and not-null coordinates.

    
    if x:
        try:
            start=stats.split().index('location:')
            end=stats.split().index('shape:')
            location_info_list=[i+" " for i in stats.split()[start+1:end]]
            location_info_str=''.join(location_info_list)
            location_info_str=location_info_str.replace('uk/england','united kingdom').replace('uk/scotland','united kingdom').replace('uk/wales','united kingdom').replace('uk/englnd','united kingdom').replace('uk/birmingham','united kingdom').replace('uk/endland','united kingdom').replace('uk/enland','united kingdom')
            location_info_str=location_info_str.replace("("," ").replace(")"," ").replace(","," ").replace("/"," ").replace("_"," ")
            
# Identify the country:
            for country in list_of_countries:
               if country in location_info_str:
                  identified_country=country
                  
                
 #Then, if the country is identified, find the city, otherwise record unknown. If city identified, find the state (admin_name in the df_worldcities table)  
        
            if  len(identified_country)>1:
               
                for city in dict_country_city[identified_country]:
                    if city in location_info_str:
                        city_corrected=city
                    
                if  len(city_corrected)>1 :  
                    if len(df_worldcities.loc[df_worldcities['city_country']==city_corrected+" "+identified_country,'admin_name'].values)==1:
                        state_corrected=df_worldcities.loc[df_worldcities['city_country']==city_corrected+" "+identified_country,'admin_name'].values[0]
                    else:
                        state_corrected='unknown'  
                else:
                    city_corrected='unknown'
                    state_corrected='unknown'
                
            
            else:
                city_corrected='unknown'
                state_corrected='unknown'
                identified_country='unknown'
         
        
        except:
            city_corrected='unknown'
            state_corrected='unknown'
            identified_country='unknown'       
                    
# Second, clean the non-mixed entries for city:                        
    else:
         
        if state_original is not None and city_original is not None:
            try:
                city_corrected=city_original
                state_corrected=state_original
                if state_original.title() in us_state_abbr.values():
                    identified_country='united states'          
                elif state_original.title() in can_prov_abbr.values():
                    identified_country='canada'
            except:
                 identified_country= "unknown"
        else:
             identified_country= "unknown"
                
                
    #return city_corrected
#df_ufo['city_corrected'] =df_ufo.apply(lambda row: find_missing_locationinfo(row['city'],row['stats'],row['state'],dict_country_city,us_state_abbr,can_prov_abbr,df_worldcities), axis=1)

    #return state_corrected
#df_ufo['state'] =df_ufo.apply(lambda row: find_missing_locationinfo(row['city'],row['stats'],row['state'],dict_country_city,us_state_abbr,can_prov_abbr,df_worldcities), axis=1)   
    
    #return identified_country
#df_ufo['country'] =df_ufo.apply(lambda row: find_missing_locationinfo(row['city'],row['stats'],row['state'],dict_country_city,us_state_abbr,can_prov_abbr,df_worldcities), axis=1)


     return city_corrected, state_corrected, identified_country
df_ufo['city_corrected','state','country'] =df_ufo.apply(lambda row: find_missing_locationinfo(row['city'],row['stats'],row['state'],dict_country_city,us_state_abbr,can_prov_abbr,df_worldcities), axis=1)


In [28]:
# Record the unique combination of city/state/country name to be used for the identification of long/lat coordinates in the next step:

df_ufo['city_state_country']=df_ufo['city_corrected']+" "+df_ufo['state']+" "+df_ufo['country']
df_ufo.head()

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude,country,population,city_corrected,date_time_from_stats,city_state_country
0,my wife was driving southeast on a fairly popu...,chester,virginia,2019-12-12t18:43:00,light,5 seconds,occurred : 12/12/2019 18:43 (entered as : 12/...,http://www.nuforc.org/webreports/151/S151739.html,My wife was driving southeast on a fairly popu...,2019-12-22T00:00:00,37.343152,-77.408582,united states,,chester,,chester virginia united states
1,i think that i may caught a ufo on the nbc nig...,rocky hill,connecticut,2019-03-22t18:30:00,circle,3-5 seconds,occurred : 3/22/2019 18:30 (entered as : 03/2...,http://www.nuforc.org/webreports/145/S145297.html,I think that I may caught a UFO on the NBC Nig...,2019-03-29T00:00:00,41.664800,-72.639300,united states,,rocky hill,,rocky hill connecticut united states
2,i woke up late in the afternoon 3:30-4pm. i we...,NaN,unknown,NaN,NaN,NaN,occurred : 4/1/2019 15:45 (entered as : april...,http://www.nuforc.org/webreports/145/S145556.html,I woke up late in the afternoon 3:30-4pm. I w...,NaN,NaN,NaN,,,NaN,,NaN
3,i was driving towards the intersection of fall...,ottawa,ontario,2019-04-17t02:00:00,teardrop,10 seconds,occurred : 4/17/2019 02:00 (entered as : 04-1...,http://www.nuforc.org/webreports/145/S145697.html,I was driving towards the intersection of fall...,2019-04-18T00:00:00,45.381383,-75.708501,canada,,ottawa,,ottawa ontario canada
4,"in peoria arizona, i saw a cigar shaped craft ...",peoria,new york,2009-03-15t18:00:00,cigar,2 minutes,occurred : 3/15/2009 18:00 (entered as : 03/1...,http://www.nuforc.org/webreports/145/S145723.html,"In Peoria, Arizona, I saw a cigar shaped craft...",2019-04-18T00:00:00,NaN,NaN,united states,,peoria,,peoria new york united states


## Step 2:

### Having identified the city, state, and country, the latitude and longitude must be identified from a complimentary dataset (df_worldcities).

### I will also identify and save the population information wherever possible, as it might be useful later for the normalization of data.

**Note**:  
<br>
The next script is relatively slow due to the type of itteration. A faster way of doing it would be through this single line: df_new_ufo=df_ufo.merge(df_worldcities, on='city_state_country', how='left',indicator=True) , i.e. left join (df_ufo being left) on the unique location key (being:city_state_country).  The equivalent SQL query would be: query='select wc.population, wc.lat, wc.lng from df_ufo as ufo left join df_worldcities as wc on ufo.city+state+country=wc.city+state+country). However, the reason that I'm not doing it this way is that the city+state+country is not perfectly unique in the df_worldcitie table (as there are some seemingly re-enties). Removing the duplicates require certain information which I didn't have. To simplify the process, I proceeded as follows:

In [29]:
# Now that we have Country, state and city for as many records as possible, lets identify all possible lat/long coordinates from unique combination of city/state/country name:

pattern = re.compile("[a-z A-Z]")

city_state_country_list= list(df_worldcities['city_state_country'])
for ind, row in df_ufo.iterrows():

    try:
        unique_location_identifier=df_ufo.loc[ind,'city_state_country']
        if unique_location_identifier in city_state_country_list:
        
            df_ufo.loc[ind,'city_latitude']=df_worldcities.loc[df_worldcities['city_state_country']==df_ufo.loc[ind,'city_state_country'],'lat'].iloc[0]
            df_ufo.loc[ind,'city_longitude']=df_worldcities.loc[df_worldcities['city_state_country']==df_ufo.loc[ind,'city_state_country'],'lng'].iloc[0]
            df_ufo.loc[ind,'population']=df_worldcities.loc[df_worldcities['city_state_country']==df_ufo.loc[ind,'city_state_country'],'population'].iloc[0]
    except:
            df_ufo.loc[ind,'city_latitude']="unknown"
            df_ufo.loc[ind,'city_longitude']="unknown"
            df_ufo.loc[ind,'population']="unknown"
        
df_ufo.head(10)
            
            

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude,country,population,city_corrected,date_time_from_stats,city_state_country
0,my wife was driving southeast on a fairly popu...,chester,virginia,2019-12-12t18:43:00,light,5 seconds,occurred : 12/12/2019 18:43 (entered as : 12/...,http://www.nuforc.org/webreports/151/S151739.html,My wife was driving southeast on a fairly popu...,2019-12-22T00:00:00,37.3531,-77.4342,united states,22144.0,chester,,chester virginia united states
1,i think that i may caught a ufo on the nbc nig...,rocky hill,connecticut,2019-03-22t18:30:00,circle,3-5 seconds,occurred : 3/22/2019 18:30 (entered as : 03/2...,http://www.nuforc.org/webreports/145/S145297.html,I think that I may caught a UFO on the NBC Nig...,2019-03-29T00:00:00,41.6572,-72.6632,united states,20137.0,rocky hill,,rocky hill connecticut united states
2,i woke up late in the afternoon 3:30-4pm. i we...,NaN,unknown,NaN,NaN,NaN,occurred : 4/1/2019 15:45 (entered as : april...,http://www.nuforc.org/webreports/145/S145556.html,I woke up late in the afternoon 3:30-4pm. I w...,NaN,unknown,unknown,,unknown,NaN,,NaN
3,i was driving towards the intersection of fall...,ottawa,ontario,2019-04-17t02:00:00,teardrop,10 seconds,occurred : 4/17/2019 02:00 (entered as : 04-1...,http://www.nuforc.org/webreports/145/S145697.html,I was driving towards the intersection of fall...,2019-04-18T00:00:00,45.4247,-75.695,canada,989567.0,ottawa,,ottawa ontario canada
4,"in peoria arizona, i saw a cigar shaped craft ...",peoria,new york,2009-03-15t18:00:00,cigar,2 minutes,occurred : 3/15/2009 18:00 (entered as : 03/1...,http://www.nuforc.org/webreports/145/S145723.html,"In Peoria, Arizona, I saw a cigar shaped craft...",2019-04-18T00:00:00,NaN,NaN,united states,,peoria,,peoria new york united states
5,"the object has flashing lights that are green,...",kirbyville,texas,2019-04-02t20:25:00,disk,15 minutes,occurred : 4/2/2019 20:25 (entered as : 04/02...,http://www.nuforc.org/webreports/145/S145476.html,"The object has flashing lights that are green,...",2019-04-08T00:00:00,30.6772,-94.0052,united states,,kirbyville,,kirbyville texas united states
6,description is the same as washington dc event...,tucson,arizona,2019-05-01t11:00:00,unknown,5 minutes,occurred : 5/1/2019 11:00 (entered as : 5/1/1...,http://www.nuforc.org/webreports/145/S145947.html,"Description is the same as Washington, DC, eve...",2019-05-09T00:00:00,32.1545,-110.8782,united states,888486.0,tucson,,tucson arizona united states
7,apr. 10th we witnessed a very bright silvery r...,gold canyon,arizona,2019-04-10t17:00:00,circle,10 minutes,occurred : 4/10/2019 17:00 (entered as : 04/1...,http://www.nuforc.org/webreports/145/S145766.html,Apr. 10th we witnessed a very bright silvery r...,2019-04-25T00:00:00,33.3715,-111.4369,united states,11038.0,gold canyon,,gold canyon arizona united states
8,ufos report in irving texas at 2200 hrs. on or...,dallas,texas,1973-07-14t22:00:00,oval,6 minutes,occurred : 7/14/1973 22:00 (entered as : 07/1...,http://www.nuforc.org/webreports/145/S145751.html,Ufos report in Irving Texas at 2200 hrs. On o...,2019-04-25T00:00:00,32.7936,-96.7662,united states,5743938.0,dallas,,dallas texas united states
9,group of lights formation sweeping thru a nigh...,caloocan city (philippines),caloocan,2019-06-06t19:00:00,other,19:00 to 19:30,occurred : 6/6/2019 19:00 (entered as : 6/6/2...,http://www.nuforc.org/webreports/146/S146694.html,group of lights formation sweeping thru a nigh...,2019-06-07T00:00:00,14.65,120.9667,philippines,1583978.0,caloocan city,,caloocan city caloocan philippines


### Checking how many of the original missing values (16112) in city_latitude/longitude were recovered:


In [30]:
    query='select count(city) from df_ufo where city_latitude is null'
   # query='select count(city),country from df_ufo where city_latitude is null group by country order by count(city) desc'
   # query='select city_corrected, city, state,country from df_ufo where city_latitude is null and country="united kingdom"'
   # query='select city_ascii, admin_name,country from df_worldcities where country="united kingdom"'
    pysql(query)

,count(city)
0,10340


#### Finding:

   * There are still 10340 missing values. The largest missing values belong to: USA (7223),unidentified country(2683),canada(503), and UK (369). 
   
   <br>
   * I checked some of them and realized that for these entries city name is wrong (name of district, or comunity instead of a city, e.g. leyner, colorado,US, and kent,UK). That's why the location couldn't be identified and consequently the latitude/longitude coordinates can not be matched with them.
   <br>



## Step 3: 

### Let's find the missing date_times, and format them.

### Missing date_time can be extracted from stats. This field then needs to be converted into a standard format:

In [31]:
#Now, Lets read all the date/times from stats, and change format to iso 
#then check the date_time, and wherever date_time is missing copy the value of date_time_from_stats in to date_time


def missing_date_time_finder(stats,original_date_time):

        try:
            date_time_str=stats.split()[2]+"t"+stats.split()[3]
            date_time_obj=datetime.datetime.strptime(date_time_str, '%m/%d/%Yt%H:%M')
            date_time_from_stats=date_time_obj.isoformat()
        except:
            date_time_from_stats=""
        
        
        try:
            date_time_existing=original_date_time
            date_time_obj=datetime.datetime.strptime(date_time_existing, '%Y-%m-%dt%H:%M:%S')
            date_time_formatted=date_time_obj.isoformat()
        except:
            date_time_formatted=date_time_from_stats
        
        return  date_time_formatted   

df_ufo['date_time'] =df_ufo.apply(lambda row: missing_date_time_finder(row['stats'],row['date_time']), axis=1)



In [32]:
#Check the editted date_time:
query='select count(date_time) from df_ufo where date_time REGEXP "[0-9a-zA-Z]"'
pysql(query)

,count(date_time)
0,87923


   * Finding: 
   <br>

After transformation, there are 87923 not null values in the date_time. Which means still 202 values are missing. Two of them can not be recovered because their stats is null too. Those 200 records must have had a different format so that my code couldn't convert them. If time permitted, I would have tried to identify and recover those records too. 

## Step 4: 

### Now let's convert the durations into seconds:

In [33]:
# convert the text duration into seconds
def duration_converter(original_duration):
    
    try:
        numeric_duration = durations.Duration(original_duration)
        duration_in_seconds=numeric_duration.to_seconds()
    except:
        duration_in_seconds="unknown"
    return duration_in_seconds

df_ufo['duration'] =df_ufo.apply(lambda row: duration_converter(row['duration']), axis=1)
# Let's check if there is any unreasonable or negative duration:

  **Note:**  The original non-null entry in duration was equal to 84954. After converting to second, using duration method,  not-null entries of duration decreased to 58548 (missing about %30 of data!). This is because 29577 entries were recorded inaccurately (uncertain or unquantifiable) and hence  couldn't be converted using the duration method. Cleaning those records requires more time and effort which is beyond the scoope of this project.

Below you can see some sample queries used in this section:

In [36]:
# Check the result of application of duration method on duration data:
#query='select duration, duration_second from df_ufo where duration is not null'
#query='select count (duration) from df_ufo where duration is not null'
query='select count (duration) from df_ufo where duration REGEXP "[0-9]"'
#query='select duration from df_ufo where duration not REGEXP "[0-9]"'
pysql(query)

,count (duration)
0,58548


## Step 5: 

### Let's format the country entries:

In [37]:
## Converting countries to iso 3166-1


## There are 17 countries whose official name in the pycountry differs from their common name (which is prsented in my data):
## therefore to convert them to iso format I have to first substitute their CODE according to pycountry 
    
exception_countries_official_name={
    'iran':'IRN',
    'venezuela': 'VEN' ,
    'reunion':'REU',
    'russia': 'RUS' ,
    'bolivia': 'BOL' ,
    'macedonia': 'MKD' ,
    'syria': 'SYR' ,
    'brunei': 'BRU' ,
    'kosovo': 'UNK' ,
    'isle of man': 'IMN' ,
    'macau': 'MAC' ,
    'tanzania': 'TZA' ,
    'taiwan': 'TWN' ,
    'west bank': '-' ,
    'curaao': 'CUW' ,
    'moldova':'MDA',
    'vietnam':  'WNM'
}
    
    
countries = {}
for country in pycountry.countries:
        countries[country.name] = country.alpha_2

            
def country_converter(original_country):
  
        if original_country in exception_countries_official_name.keys():
            formatted_country=exception_countries_official_name[original_country]
        else:
            title_country=original_country.title()
            formatted_country=countries.get(title_country, 'unknown')  
            
        return formatted_country


df_ufo['country_iso'] =df_ufo.apply(lambda row: country_converter(row['country']), axis=1)    

In [38]:
query='select count (report_link) from df_ufo where country_iso!="unknown"'
pysql(query)

,count (report_link)
0,85048


## Step 6:  

### Finally, let's drop the useless fields, replace the leftover null values by "unknown", and save the transformed data into a csv file:

In [ ]:
# Drop the fields that are not useful in data analysis
df_ufo_clean=df_ufo.drop([ 'summary','city','report_link','text','posted','city_state_country','date_time_from_stats'], axis=1)

In [ ]:
#Overwriting the Null values in duration and shape field:
df_ufo_clean.loc[df_ufo_clean["state"].isnull(), 'state'] = 'unknown'
df_ufo_clean.loc[df_ufo_clean["shape"].isnull(), 'shape'] = 'unknown'
df_ufo_clean.loc[df_ufo_clean["duration"].isnull(), 'duration'] = 'unknown'
df_ufo_clean.loc[df_ufo_clean["date_time"].isnull(), 'date_time'] = 'unknown'
df_ufo_clean.loc[df_ufo_clean["city_latitude"].isnull(), 'city_latitude'] = 'unknown'
df_ufo_clean.loc[df_ufo_clean["city_longitude"].isnull(), 'city_longitude'] = 'unknown'
df_ufo_clean.loc[df_ufo_clean["country"].isnull(), 'country'] = 'unknown'
df_ufo_clean.loc[df_ufo_clean["population"].isnull(), 'population'] = 'unknown'
df_ufo_clean.loc[df_ufo_clean["city_corrected"].isnull(), 'city_corrected'] = 'unknown'
df_ufo_clean.loc[df_ufo_clean["country_iso"].isnull(), 'country_iso'] = 'unknown'
df_ufo_clean.head(20)

In [ ]:
#Save the cleaned data (final version)
df_ufo_clean.to_csv('filename')